In [1]:
import spacy
import medspacy
from medspacy.visualization import visualize_ent, visualize_dep

# Overview
In this notebook, we'll look at how to extract clinical concepts and attributes from text.
- Target matching
- Context analysis

In [2]:
with open("./discharge_summary.txt") as f:
    text = f.read()

In [3]:
nlp = spacy.load("en_core_web_sm", disable=["ner"])

# Target extraction
In this step, we'll write rules to extract the main concepts we're interested in.

In this example, we'll use two utilities provided in `medspacy.ner` for rule-based matching: the `TargetMatcher` and `TargetRule`. However, you can use any spaCy components for adding spans to `doc.ents`, including pre-trained NER models or other [spaCy rule-based matching components](https://spacy.io/usage/rule-based-matching/).

## Target concepts
In our text, we'll extract the following concepts:
- Diagnoses 
- Medications

In addition, we'll show a few examples of how to add a custom spaCy attribute to a target rule to add an ICD-10 diagnosis code as an attribute of an entity.

In [4]:
from medspacy.ner import TargetMatcher, TargetRule

In [5]:
target_matcher = TargetMatcher(nlp)

In [6]:
nlp.add_pipe(target_matcher)

In [7]:
target_rules1 = [
    TargetRule(literal="abdominal pain", category="PROBLEM"),
    TargetRule("stroke", "PROBLEM"),
    TargetRule("hemicolectomy", "TREATMENT"),
    TargetRule("Hydrochlorothiazide", "TREATMENT"),
    TargetRule("colon cancer", "PROBLEM"),
    TargetRule("radiotherapy", "PROBLEM",
              pattern=[{"LOWER": "xrt"}]),
    TargetRule("metastasis", "PROBLEM"),
    
]

In [8]:
target_matcher.add(target_rules1)

In [9]:
doc = nlp(text)

In [10]:
visualize_ent(doc)

In [11]:
for ent in doc.ents:
    print(ent, ent.label_, ent._.target_rule.literal, sep="  |  ")
    print()

Hydrochlorothiazide  |  TREATMENT  |  Hydrochlorothiazide

Abdominal pain  |  PROBLEM  |  abdominal pain

stroke  |  PROBLEM  |  stroke

abdominal pain  |  PROBLEM  |  abdominal pain

metastasis  |  PROBLEM  |  metastasis

Colon cancer  |  PROBLEM  |  colon cancer

hemicolectomy  |  TREATMENT  |  hemicolectomy

XRT  |  PROBLEM  |  radiotherapy

stroke  |  PROBLEM  |  stroke

abdominal pain  |  PROBLEM  |  abdominal pain

abdominal pain  |  PROBLEM  |  abdominal pain



## Adding custom attributes
One of the most powerful functionalities of spaCy is the ability to add [custom attributes](https://spacy.io/usage/processing-pipelines#custom-components-attributes) to spaCy objects (`Doc`, `Span`, or `Token`). These custom attributes are accessed through `obj._....`. As we'll see in later steps, medSpaCy adds several of these attributes in other attributes like `context` or `sectionizer`. 

But it is sometimes useful to include a custom attribute as part of the target matching rule, rather than needing to build a separate component to add it. The `TargetRule` can also include a value for these attributes in the `attributes` argument. 

For example, let's say we want to map certain entities to [ICD-10 diagnosis codes](https://www.cdc.gov/nchs/icd/icd10cm.htm). One way we can do this is to include the diagnosis codes for concepts in our knowledge base. For example, **"Type II Diabetes"** can be mapped to **E11.9**". We can add this to entities by first registering the extension for the `Span` class:


In [12]:
from spacy.tokens import Span

In [13]:
Span.set_extension("icd10", default="")

We can now include ICD-10 code values in the target rules. We'll map **"Type II Diabetes Mellitus"** to **"E11.9"** and **"Hypertension"** to **"I10"**:

In [14]:
target_rules2 = [
    TargetRule("Type II Diabetes Mellitus", "PROBLEM", 
              pattern=[
                  {"LOWER": "type"},
                  {"LOWER": {"IN": ["2", "ii", "two"]}},
                  {"LOWER": {"IN": ["dm", "diabetes"]}},
                  {"LOWER": "mellitus", "OP": "?"}
              ],
              attributes={"icd10": "E11.9"}),
    TargetRule("Hypertension", "PROBLEM",
              pattern=[{"LOWER": {"IN": ["htn", "hypertension"]}}],
              attributes={"icd10": "I10"}),
    
    
]

In [15]:
target_matcher.add(target_rules2)

In [16]:
doc = nlp(text)

Now, whenever one of these rules results in a match, the ICD-10 value can be accessed in `ent._.icd10`:

In [17]:
for ent in doc.ents:
    if ent._.icd10 != "":
        print(ent, ent._.icd10)

type 2 dm E11.9
Type II Diabetes Mellitus E11.9
Hypertension I10
Type 2 DM E11.9
HTN I10


# Context
Clinical text often contains mentions of concepts which the patient did not actually experience. For example:

- "There is *no evidence of* **pneumonia**"
- "*Mother* with **breast cancer**"
- "Patient presents for *r/o* **COVID-19**"

In all of these instances, we need to use the contextual clues around the entity to assert attributes like negation, experiencer, and uncertainty.

One method for this is the [ConText algorithm](https://www.sciencedirect.com/science/article/pii/S1532046409000744). ConText links target entities like problems with semantic modifiers like those shown above. The medSpaCy implementation of ConText is found in `medspacy.context`.


In [18]:
from medspacy.context import ConTextComponent, ConTextItem

In [19]:
context = ConTextComponent(nlp, rules="default")

In [20]:
nlp.add_pipe(context)

In [21]:
nlp.pipe_names

['tagger', 'parser', 'target_matcher', 'context']

In [22]:
doc = nlp("Mother with stroke at age 82.")

We can use medSpaCy visualizers from the module `medspacy.visualization` to show the target/modifiers in text. `visualize_dep` shows arrows targets to show which concepts are modified by the semantic modifiers:

In [23]:
visualize_ent(doc)

In [24]:
visualize_dep(doc)

In [25]:
short_doc = nlp("Colon cancer diagnosed in 2012")

We can add a new rule using the `ConTextItem` class:

In [26]:
item_data = [
    ConTextItem("diagnosed in <YEAR>", "HISTORICAL", 
                rule="BACKWARD", # Look "backwards" in the text (to the left)
               pattern=[
                   {"LOWER": "diagnosed"},
                   {"LOWER": "in"},
                   {"LOWER": {"REGEX": "^[\d]{4}$"}}
               ])
]

In [27]:
context.add(item_data)

In [28]:
short_doc = nlp("Colon cancer diagnosed in 2012")

In [29]:
visualize_ent(short_doc)

In [30]:
visualize_dep(short_doc)

In addition to linking targets and modifiers, ConText will also set attributes for each entity:

In [31]:
for ent in doc.ents:
    if any([ent._.is_negated, ent._.is_uncertain, ent._.is_historical, ent._.is_family, ent._.is_hypothetical, ]):
        print("'{0}' modified by {1} in: '{2}'".format(ent, ent._.modifiers, ent.sent))
        print()

'stroke' modified by (<TagObject> [Mother, FAMILY],) in: 'Mother with stroke at age 82.'

